In [207]:
from playwright.async_api import async_playwright
import asyncio
import os
import pandas as pd
from pathlib import Path
import openpyxl
import tabula

In [23]:
# Určení proměnných
# url = odkaz na stránky se stažením souborů
# download_path = cesta kam se mají uložit

url = "https://mpo.gov.cz/cz/podnikani/zivnostenske-podnikani/statisticke-udaje-o-podnikatelich/pocty-podnikatelu-dle-obcanstvi-podnikajicich-v-ceske-republice--151024/"
download_path = "./Download"

In [50]:
# Define the Playwright ~ Chromium download function

async def fetch_and_download_files(url, download_path):
    # Make sure the download path exists
    if not os.path.exists(download_path):
        os.makedirs(download_path)
    
    print("Done 0")
    async with async_playwright() as pw:
        browser = await pw.chromium.launch(headless=True)
        page = await browser.new_page()

        # Set the download behavior
        page.on('download', lambda download: download.save_as(os.path.join(download_path, download.suggested_filename)))

        await page.goto(url)
        print("Done 1")

        # Wait for the page to be fully loaded
        await page.wait_for_load_state('load')
        print("Done 2")

        # Wait for the download links to appear
        await page.wait_for_selector('a.download')
        print("Done 3")

        # Get all 'a' elements with the class 'download'
        download_links = await page.query_selector_all('a.download')
        print("Done 4")

        # Click each download link and wait for the file to be downloaded
        if download_links:
            d_counts = 0
            for link in download_links:
                href = await link.get_attribute('href')
                #print(f"Attempting to download: {href}")
                # Click the link to start the download
                await link.click()
                print("Clicking the download link")
                d_counts =+ 1
                # Wait for the download to complete (you can adjust this wait time if necessary)
                await page.wait_for_timeout(500)  # Adjust if needed to allow the download to finish
        else:
            print("No download links found")
        
        print(f"Done with {d_counts} downloads")
        await browser.close()


In [ ]:
# -------------------
# RUN download script
# -------------------
await fetch_and_download_files(url, download_path)

In [19]:
# Zjistit, kolik se stáhlo souborů a vypsat jména

folder_path = Path(download_path)

processed = 0
filenames = []
# Iterate over all files in the folder
for file in folder_path.iterdir():
    if file.is_file():  # Check if it's a file
        processed += 1
        filenames.append(file.name)
        #print(f"Processing file: {file.name}")
        # Add your processing logic here

print(f"Ve složce je {processed} souborů.")
print(f"Toto jsou jejich jména: {filenames}")

Ve složce je 17 souborů.
Toto jsou jejich jména: ['2024_3.xlsx', '2019_1.xlsx', '2016_4.xlsx', '2017_1.xlsx', '2024_2.xlsx', '2018_4.xlsx', '2016_2.pdf', '2018_3.xlsx', '2018_2.xlsx', '2017_4.xlsx', '2018_1.xlsx', '2020_1.xlsx', '2017_2.xlsx', '2019_3.xlsx', '2024_1.xlsx', '2019_2.xlsx', '2017_3.xlsx']


In [141]:
# Zkusmo otevřít jeden .xlsx soubor a uložit ho jako .csv
file = "2016_4"
file_path = (f"{download_path}/{file}.xlsx")
df = pd.read_excel(file_path, engine='openpyxl')

# Save as CSV
csv_path = (f"{download_path}_CSV/{file}.csv")
df.to_csv(csv_path, index=False)
print(f"File converted to: {csv_path}")


File converted to: ./Download_CSV/2016_4.csv


In [199]:
# Otevřeme v pandasu a zahodíme 4 zbytečné řádky + resetujeme index

df = pd.read_csv(csv_path, header=None)  # Read without headers
dff = df.iloc[4:].reset_index(drop=True)  # Drop first 4 rows and reset the index
dff.columns = dff.iloc[0]  # Set the 5th row (which is now the first row after dropping) as column names
dff = dff.drop(0).reset_index(drop=True)  # Drop the row that has been used as column names

In [201]:
print(df.shape)
print(dff.shape)
print(dff.head(2))

(168, 19)
(163, 19)
0    Občanství  Kód Kód zn. Kód zn. Hlavní město Praha Středočeský kraj  \
0  Afghánistán  004      AF     AFG                 15                7   
1      Albánie  008      AL     ALB                 11                8   

0 Jihočeský kraj Plzeňský kraj Karlovarský kraj Ústecký kraj Liberecký kraj  \
0              0             0                1            6              2   
1              4             1                1            2              1   

0 Královéhradecký kraj Pardubický kraj Kraj Vysočina Jihomoravský kraj  \
0                    0               0             1                13   
1                    2               1             1                 2   

0 Olomoucký kraj Moravskoslezský kraj Zlínský kraj Součet  
0              4                    2            0     51  
1              1                    1            4     40  


In [204]:
# Základní přehled
print(f"Dataframe má tvar: {dff.shape}")
print(f"Indexy: {dff.index}")
print(f"Sloupce se jmenují: {dff.columns}")


Dataframe má tvar: (163, 19)
Indexy: RangeIndex(start=0, stop=163, step=1)
Sloupce se jmenují: Index(['Občanství', 'Kód', 'Kód zn.', 'Kód zn.', 'Hlavní město Praha',
       'Středočeský kraj', 'Jihočeský kraj', 'Plzeňský kraj',
       'Karlovarský kraj', 'Ústecký kraj', 'Liberecký kraj',
       'Královéhradecký kraj', 'Pardubický kraj', 'Kraj Vysočina',
       'Jihomoravský kraj', 'Olomoucký kraj', 'Moravskoslezský kraj',
       'Zlínský kraj', 'Součet'],
      dtype='object', name=0)


In [211]:
# Automatizace převodu celé složky .XLSX na správné .CSV

download_path = "./Download"
export_path = "./Download_CSV"


folder_path = Path(download_path)

files = list(folder_path.iterdir())
print(files)

processed = 0
#filenames = []
# Iterate over all files in the folder
for file in folder_path.iterdir():
    if file.is_file():  # Check if it's a file
        processed += 1
        #filenames.append(file.name)
        #print(f"Processing file: {file.name}")

        # Zkusmo otevřít jeden .xlsx soubor a uložit ho jako .csv
        file_path = (f"{download_path}/{file.name}")
        df = pd.read_excel(file_path, engine='openpyxl')

        # Save as CSV
        newname = (file.name).replace(".xlsx", "")
        csv_path = (f"{export_path}/{newname}.csv")
        df.to_csv(csv_path, index=False)
        print(f"Soubor {csv_path} byl úspěšně uložen", end="\n")

print(f"Ve složce je {processed} souborů.")
#print(f"Toto jsou jejich jména: {filenames}")

SyntaxError: f-string: empty expression not allowed (4193627854.py, line 30)

In [ ]:
'''
------------------------------ THE END ---------------------------------
'''

In [ ]:
def process_locked_xlsx_files(input_folder, output_folder):
    """
    Process all .xlsx files in the input folder:
    - Reads locked sheets.
    - Saves them as .csv files.
    - Loads the data into pandas DataFrames for further processing.

    Parameters:
    input_folder (str): Path to the folder containing .xlsx files.
    output_folder (str): Path to the folder to save the .csv files.

    Returns:
    list: A list of pandas DataFrames.
    """
    # Ensure the output folder exists
    Path(output_folder).mkdir(parents=True, exist_ok=True)
    
    # List to store DataFrames
    dataframes = []
    
    # Iterate over all .xlsx files in the input folder
    for file in Path(input_folder).glob("*.xlsx"):
        try:
            print(f"Processing file: {file.name}")
            
            # Load the workbook
            wb = load_workbook(filename=file, data_only=True)
            
            # Get the first sheet (adjust logic if a specific sheet is needed)
            sheet_name = wb.sheetnames[0]
            sheet = wb[sheet_name]
            
            # Extract sheet data
            data = [[cell.value for cell in row] for row in sheet.iter_rows()]
            
            # Convert to pandas DataFrame
            df = pd.DataFrame(data[1:], columns=data[0])  # Assumes first row is the header
            dataframes.append(df)
            
            # Save as CSV
            csv_file = Path(output_folder) / f"{file.stem}.csv"
            df.to_csv(csv_file, index=False)
            print(f"Saved CSV: {csv_file}")
        
        except Exception as e:
            print(f"Error processing {file.name}: {e}")
    
    return dataframes

# Define input and output folders
input_folder = "Download"  # Path to your .xlsx files folder
output_folder = "Processed"  # Folder to save .csv files

# Process files
dfs = process_locked_xlsx_files(input_folder, output_folder)

# Merge all DataFrames if needed
if dfs:
    merged_df = pd.concat(dfs, ignore_index=True)
    print("Merged DataFrame:")
    print(merged_df.head())

    # Save the merged DataFrame as a CSV
    merged_csv_path = Path(output_folder) / "merged_data.csv"
    merged_df.to_csv(merged_csv_path, index=False)
    print(f"Merged data saved to {merged_csv_path}")
